In [2]:
import os

In [3]:
%pwd

'c:\\personal projects\\chick_disease_class\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\personal projects\\chick_disease_class'

In [6]:
from dataclasses import dataclass
from pathlib import Path



## creating an entity i.e, a custom return type using entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from CNNclassifier.constants import *

In [8]:
from CNNclassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    ## using the entity created above as the return type
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        ##storing the data ingestion configuration from the config.yaml into config
        config = self.config.data_ingestion

        ## creating the root directory
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from CNNclassifier import logger
from CNNclassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"downloaded file from {self.config.source_URL} to {self.config.local_data_file}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

    

In [14]:
## Pipelines

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-08 10:06:44,989: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-08 10:06:44,989: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-08 10:06:44,989: INFO: common: created directory at: artifacts]
[2024-10-08 10:06:44,989: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-08 10:06:44,989: INFO: 273533491: file already exists of size: ~ 11345 KB]


In [15]:
## reran as zip file was corrupted
data_ingestion.extract_zip_file()